In [2]:
from google import genai
import os
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv('GOOGLE_API_KEY')
client = genai.Client(api_key=API_KEY)

In [10]:
import requests

url = "https://upload.wikimedia.org/wikipedia/commons/3/38/WING_IT%21_-_Blender_Open_Movie-full_movie.webm"
path = "wingit.webm"

response = requests.get(url, stream=True)

with open(path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

print("Download complete.")



Download complete.


In [11]:
video_file = client.files.upload(file=path)

In [ ]:
import time
# Wait until the uploaded video is available
while video_file.state.name == "PROCESSING":
  print('.', end='')
  time.sleep(5)
  video_file = client.files.get(name=video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Display some of the video content
cap = cv2.VideoCapture(path)
frame_number = 1000
for _ in range(frame_number):
    ret, frame = cap.read()
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

plt.imshow(frame_rgb)
plt.axis('off')
plt.show()

cap.release()
     

In [14]:
system_prompt = "You should provide a quick 2 or 3 sentence summary of what is happening in the video."

In [ ]:
from google.genai import types

MODEL_ID="gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}
response = client.models.generate_content(
    model=f"models/{MODEL_ID}",
    contents=[
        "Summarise this video please.",
        video_file
        ],
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        ),
    )
print(response.text)